In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
def auto_count(months):
    '''
    function that takes in a list of csv file name strings
    and returns a dataframe of counted duty and lunch shifts
    '''
    
    def read(csv, weeks):
        '''
        function that opens csv as a dataframe,
        cleans data to return a list of shifts + empty strings
        '''
        month = pd.read_csv(csv)
        month.columns = month.iloc[0]
        month = month.iloc[::2]
        month = month.iloc[1:,:]
        return month
    
    def count(weeks, duty, lunch):
        lunchpat = r"Lunch 12[ ]*-[ ]*1[pm| ]*:[ ]*([A-Za-z\d]+)"
        dutypats = [r"4:30[pm| ]*-[ ]*9[pm| ]*:[ ]*([A-Za-z\d]+)", 
                       r"9[am| ]*-[ ]*1[pm| ]*:[ ]*([A-Za-z\d]+)", 
                       r"1[pm| ]*-[ ]*5[pm| ]*:[ ]*([A-Za-z\d]+)", 
                       r"5[pm| ]*-[ ]*9[pm| ]*:[ ]*([A-Za-z\d]+)"]
        for day in weeks:
            if day != "":
                if re.match(lunchpat, day) != None:
                    add_count(re.findall(lunchpat, day)[0], lunch)
                for dut in list(dutypats):
                    if re.match(dut, day) != None:
                        add_count(re.findall(dut, day)[0], duty)
                        break
                        
    def add_count(name, counts):
        try:
            counts[name] += 1
        except:
            counts[name] = 1
    
    duty = {}
    lunch = {}
    monthdfs = []
    weeks = []
    
    for i in months:
        monthdfs.append(read(i, weeks))
        
    monthdf = pd.concat(monthdfs)
    duties = sum(monthdf.values.tolist(), weeks)
    duties = "\n".join(str(x) for x in duties if pd.isna(x) == False)
    duties = duties.split("\n")
    
    count(duties, duty, lunch)
    
    d = pd.DataFrame.from_dict(duty, orient = "index")
    d = d.rename(columns={d.columns[0]: 'Duty'})
    l = pd.DataFrame.from_dict(lunch, orient = "index")
    l = l.rename(columns={l.columns[0]: 'Lunch'})
    
    return pd.concat([d, l], axis = 1).fillna(0).astype(int)

In [3]:
cwd = os.getcwd()
files = os.listdir(cwd)

months = [i for i in files if i[0] == "C"]
    
ds = auto_count(months)

In [4]:
ds["Total Hours"] = (ds["Duty"] * 4) + ds["Lunch"]
ds.sort_values("Total Hours", ascending = False)

,Duty,Lunch,Total Hours
student2,17,1,69
student23,15,8,68
student27,15,6,66
student18,15,6,66
student26,14,9,65
student20,15,5,65
student25,15,5,65
student9,14,8,64
student3,14,8,64
student28,14,7,63
